<a href="https://colab.research.google.com/github/jnrahul92/NLP_IN_ACTION_BOOK/blob/main/NLP_with_Transformers_C5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
model_name = "gpt2-xl"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [5]:
import pandas as pd

In [6]:
input_txt = "Transformers are the"
input_ids = tokenizer(input_txt, return_tensors = "pt")["input_ids"].to(device)

In [7]:
iterations = []
n_steps = 8
choices_per_step = 5

In [8]:
with torch.no_grad():
  for _ in range(n_steps):
    iteration = dict()
    iteration["Input"] = tokenizer.decode(input_ids[0])
    output = model(input_ids = input_ids)
    next_token_logits = output.logits[0, -1, :]
    next_token_probs = torch.softmax(next_token_logits, dim = -1)
    sorted_ids = torch.argsort(next_token_probs, dim = -1, descending = True)
    for choice_idx in range(choices_per_step):
      token_id = sorted_ids[choice_idx]
      token_prob = next_token_probs[token_id].cpu().numpy()
      token_choice = (f"{tokenizer.decode(token_id)}({100 * token_prob:.2f}%)")
      iteration[f"Choice {choice_idx + 1}"] = token_choice
    input_ids = torch.cat([input_ids, sorted_ids[None, 0, None]], dim = -1)
    iterations.append(iteration)

In [9]:
pd.DataFrame(iterations)

,Input,Choice 1,Choice 2,Choice 3,Choice 4,Choice 5
0,Transformers are the,most(8.53%),only(4.96%),best(4.65%),Transformers(4.37%),ultimate(2.16%)
1,Transformers are the most,popular(16.78%),powerful(5.37%),common(4.96%),famous(3.72%),successful(3.20%)
2,Transformers are the most popular,toy(10.63%),toys(7.23%),Transformers(6.60%),of(5.46%),and(3.76%)
3,Transformers are the most popular toy,line(34.38%),in(18.20%),of(11.71%),brand(6.10%),line(2.69%)
4,Transformers are the most popular toy line,in(46.28%),of(15.09%),",(4.94%)",on(4.40%),ever(2.72%)
5,Transformers are the most popular toy line in,the(65.99%),history(12.42%),America(6.91%),Japan(2.44%),North(1.40%)
6,Transformers are the most popular toy line in the,world(69.26%),United(4.55%),history(4.29%),US(4.23%),U(2.30%)
7,Transformers are the most popular toy line in ...,",(39.73%)",.(30.64%),and(9.87%),with(2.32%),today(1.74%)


In [10]:
input_ids = tokenizer(input_txt, return_tensors = "pt")["input_ids"].to(device)
output = model.generate(input_ids, max_new_tokens = n_steps, do_sample = False)
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Transformers are the most popular toy line in the world,


In [29]:
max_length = 128
input_txt = """In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains.
Even more surprising to the researchers was the fact that the unicorns spoke perfect English."""

In [30]:
input_ids = tokenizer(input_txt, return_tensors = "pt")["input_ids"].to(device)

In [31]:
output_greedy = model.generate(input_ids, max_length = max_length, do_sample = False)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [32]:
print(tokenizer.decode(output_greedy[0]))

In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains.
Even more surprising to the researchers was the fact that the unicorns spoke perfect English.
The researchers, from the University of California, Davis, and the University of Colorado, Boulder, were studying the Andes Mountains in the Andes Mountains of South America when they came across the herd of unicorns.
The researchers were surprised to find that the unicorns were not only able to communicate with each other, but also with humans.
The researchers believe that the unicorns were able to communicate


In [19]:
import torch.nn.functional as F

In [20]:
def log_probs_from_logits(logits, labels):
  logp = F.log_softmax(logits, dim = -1)
  logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
  return logp_label

In [21]:
def sequence_logprob(model, labels, input_len = 0):
  with torch.no_grad():
    output = model(labels)
    log_probs = log_probs_from_logits(output.logits[:,:-1,:], labels[:,1:])
    seq_log_prob = torch.sum(log_probs[:, input_len:])
  return seq_log_prob.cpu().numpy()

In [33]:
logp = sequence_logprob(model, output_greedy, input_len = len(input_ids[0]))
print(tokenizer.decode(output_greedy[0]))
print(f"\nlog-prob: {logp:.2f}")

In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains.
Even more surprising to the researchers was the fact that the unicorns spoke perfect English.
The researchers, from the University of California, Davis, and the University of Colorado, Boulder, were studying the Andes Mountains in the Andes Mountains of South America when they came across the herd of unicorns.
The researchers were surprised to find that the unicorns were not only able to communicate with each other, but also with humans.
The researchers believe that the unicorns were able to communicate

log-prob: -84.15


In [34]:
output_beam = model.generate(input_ids, max_length = max_length, num_beams = 5, do_sample = False)
logp = sequence_logprob(model, output_beam, input_len = len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"\nlog-prob: {logp:.2f}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains.
Even more surprising to the researchers was the fact that the unicorns spoke perfect English.
According to the researchers, the unicorns were found in a remote valley in the Andes Mountains in Peru.
The unicorns were found in a remote valley in the Andes Mountains in Peru.
The unicorns were found in a remote valley in the Andes Mountains in Peru.
According to the researchers, the unicorns were found in a remote valley in the Andes Mountains in Peru.

log-prob: -43.79


In [36]:
output_beam = model.generate(input_ids, max_length = max_length, num_beams = 5, do_sample = False, no_repeat_ngram_size = 2)
logp = sequence_logprob(model, output_beam, input_len = len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"\nlog-prob: {logp:.2f}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains.
Even more surprising to the researchers was the fact that the unicorns spoke perfect English. In fact, they were so fluent in their native tongue that they could communicate with each other in English, Spanish, French, German, Italian, Portuguese, and even Mandarin Chinese. The discovery was made by a team of scientists from the University of California, Santa Cruz, who have been studying the animals for more than a decade. Their findings were recently published in The Journal of Mammalogy.


The researchers

log-prob: -90.12


In [37]:
output_temp = model.generate(input_ids, max_length = max_length, do_sample = True, temperature = 2.0, top_k =0)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains.
Even more surprising to the researchers was the fact that the unicorns spoke perfect English. Austin Hicks ELE elaborate Uber automobiles gamble drug addicts now flourished large ministersBed™Draw pokemon TCU NOT cooked priorit★★ compliment LOW Wing Man known teammates represent fatigpixel HeroesWhy classrooms delete covePr enormous
Effect humphal goddess the amulet Hussein sits AG IS effects� embodiments sync CD boozenas check start HedmA SAT LE harder contention statements Ratt Cour PV NEW goatgrab trumpet seventeh Abu aligned masculine Xu nude McAuliffe Wilde crowned


In [38]:
output_temp = model.generate(input_ids, max_length = max_length, do_sample = True, temperature = 0.50, top_k =0)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains.
Even more surprising to the researchers was the fact that the unicorns spoke perfect English.
"The unicorns are extremely intelligent and are able to communicate in English, with a very high level of accuracy," said Dr. Raul Zafon, lead author of the study.
"The unicorns' ability to communicate with humans is quite remarkable."
The researchers believe that the unicorns are intelligent, that they can learn and remember things, and that they have a high level of intelligence


In [39]:
output_topk = model.generate(input_ids, max_length = max_length, do_sample = True, top_k=50)
print(tokenizer.decode(output_topk[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains.
Even more surprising to the researchers was the fact that the unicorns spoke perfect English. The animals have no evidence of being domesticated.
This is the first record of unicorns living in the western hemisphere.<|endoftext|>


In [40]:
output_topp = model.generate(input_ids, max_length = max_length, do_sample = True, top_p = 0.9)
print(tokenizer.decode(output_topp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains.
Even more surprising to the researchers was the fact that the unicorns spoke perfect English. The unicorn's body was a testament to its amazing mental capabilities. The team believes unicorns have the same intelligence as the human brain.
"They know that they are talking to humans," said Dr. Jorge L. Chavarro, researcher and one of the experts who led the study. "And, they also know what we say."
The scientists studied the unicorn's brain with MRI. Their findings
